In [3]:
import sys
sys.path.append('../RecSysRep/')

In [4]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
URM_all_dataframe, ICM_genre_all_dataframe, ICM_subgenre_all_dataframe, ICM_channel_all_dataframe, ICM_event_all_dataframe = ld.getDataframes()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [5]:
# ICM_genre_all -> toarray -> DATAFRAME
URM_all_CSR = URM_all.tocsr()
ICM_genre_all_CSR = ICM_genre_all.tocsr()
ICM_subgenre_all_CSR = ICM_subgenre_all.tocsr()
ICM_channel_all_CSR = ICM_channel_all.tocsr()

### Create array of indices of all TV shows watched by one user

In [6]:
user_id = 0
start = URM_all_CSR.indptr[user_id]
end = URM_all_CSR.indptr[user_id+1]
watched_movies = URM_all_CSR.indices[start:end]

print(watched_movies)

[   53   209   223   249   435   460   652   960   973  1103  1215  1267
  1418  1455  1461  1496  1782  1964  2083  2307  2489  2678  2844  2886
  2974  3065  3136  3140  3473  3492  3544  3619  3830  3942  4081  4292
  4365  4438  4522  4571  4582  4607  5051  5134  5273  5299  5300  5403
  5414  5665  5793  5925  5951  6006  6112  6170  6177  6214  6233  6300
  6488  6641  6933  6975  7015  7089  7099  7407  7645  7884  7899  7919
  7971  7975  8011  8111  8194  8213  8371  8567  8709  8747  8911  8942
  9308  9426  9482  9508  9610  9618  9734  9908  9959  9969  9992 10197
 10382 10418 10624 10865 10866 10954 11074 11282 11340 11394 11430 11507
 11564 11576 11607 11870 12186 12281 12296 12310 12311 12371 12390 12707
 12788 13237 13281 13848 13865 14044 14181 14227 14358 14400 14468 14477
 14485 14520 14952 15064 15082 15119 15290 15503 15507 15514 15551 15624
 15712 15740 15844 16028 16136 16364 16520 16549 16731 16751 16846 16897
 16932 17000 17001 17294 17297 17330 17428 17565 17

### Get GenreID for each TV show

In [7]:
user_genres = ICM_genre_all_dataframe.loc[ICM_genre_all_dataframe['ItemID'].isin(watched_movies)]
print(user_genres)

       ItemID  GenreID  Match
50         53        4    1.0
206       209        6    1.0
219       223        7    1.0
247       249        7    1.0
434       435        3    1.0
...       ...      ...    ...
17494   17565        7    1.0
17555   17625        7    1.0
17719   17788        6    1.0
17758   17827        3    1.0
17789   17858        6    1.0

[162 rows x 3 columns]


In [8]:
mode = user_genres['GenreID'].mode()[0]
print("Most watched genre by user " + str(user_id) + ":\n" + str(mode))

Most watched genre by user 0:
7


In [9]:
variance = len(user_genres.loc[user_genres['GenreID'] != mode])
perc_nonfav = variance/len(user_genres) * 100

print("Percentage of watched TV shows that are not the user's favourite genre: " + str(perc_nonfav))

Percentage of watched TV shows that are not the user's favourite genre: 61.72839506172839


# Interesting part

In [10]:
feature_CSR = ICM_genre_all_CSR

### Get matrix with number of TV shows watched per genre per user

In [11]:
import pandas as pd
matrix_usergenres = URM_all_CSR * feature_CSR
df_usergenres = pd.DataFrame(matrix_usergenres.toarray())
df_usergenres

,0,1,2,3,4,5,6,7
0,7.0,1.0,2.0,20.0,7.0,16.0,47.0,62.0
1,20.0,4.0,7.0,222.0,92.0,42.0,82.0,138.0
2,10.0,0.0,0.0,53.0,6.0,18.0,20.0,31.0
3,5.0,1.0,1.0,8.0,3.0,0.0,26.0,7.0
4,30.0,8.0,5.0,232.0,19.0,38.0,127.0,146.0
...,...,...,...,...,...,...,...,...
13645,197.0,5.0,3.0,216.0,7.0,83.0,25.0,78.0
13646,19.0,2.0,1.0,29.0,83.0,25.0,18.0,41.0
13647,9.0,6.0,3.0,54.0,44.0,30.0,31.0,76.0
13648,4.0,0.0,0.0,41.0,39.0,12.0,30.0,53.0


### Get matrix with percentage of watched TV shows that are the user's favourite genre

In [164]:
df_nonfav = (df_usergenres.max(axis=1)/df_usergenres.sum(axis=1)) * 100
print(df_nonfav)

0        38.271605
1        36.573311
2        38.405797
3        50.980392
4        38.347107
           ...    
13645    35.179153
13646    38.073394
13647    30.039526
13648    29.608939
13649    27.051672
Length: 13650, dtype: float64


In [165]:
df_nonfav.describe()

count    13650.000000
mean        35.056996
std          7.708112
min         17.647059
25%         29.520540
50%         33.468565
75%         39.187810
max         90.534351
dtype: float64

### Get entropy

In [166]:
import numpy as np
# print(df_usergenres.idxmax(axis=1))
total_views_per_user = df_usergenres.sum(axis=1)
# print(total_views_per_user)
df_prob = df_usergenres.div(total_views_per_user, axis=0)
df_ent = df_prob.mul(-np.log2(df_prob))
df_ent = df_ent.fillna(0)
df_entropy = df_ent.sum(axis=1)

df_entropy

0        2.265977
1        2.370114
2        2.272393
3        2.099338
4        2.260168
           ...   
13645    2.180496
13646    2.430868
13647    2.546621
13648    2.301713
13649    2.440183
Length: 13650, dtype: float64

In [167]:
df_entropy.describe()

count    13650.000000
mean         2.345460
std          0.166769
min          0.676733
25%          2.250559
50%          2.367012
75%          2.461359
max          2.895424
dtype: float64

### Usage

In [16]:
from Basics.Entropy import getMeanEntropy

print(getMeanEntropy(ICM_channel_all_CSR, URM_all_CSR).describe())

count    13650.000000
mean         5.661848
std          0.547700
min          1.736641
25%          5.372767
50%          5.709816
75%          6.026688
max          6.971522
dtype: float64
